# Twitter Airline BERT LGBM

In [1]:
!pip install transformers > /dev/null

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import os
import torch
import random
import matplotlib.pyplot as plt
import transformers
from transformers import BertTokenizer
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
#from xgboost import XGBRegressor
import lightgbm as lgb
from tqdm import tqdm
tqdm.pandas()

In [3]:
data0 = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')
display(data0)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [4]:
data=data0[['airline_sentiment','text']]
data=data.dropna()
data['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [5]:
Name0=data['airline_sentiment'].unique().tolist()
Name=sorted(Name0)
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 
data['airline_sentiment']=data['airline_sentiment'].map(normal_mapping)

In [6]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        #self.model_name = '../input/bert-base-uncased'  # Inet-not-connect
        self.model_name = 'bert-base-uncased'          # Inet-connect
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128

    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

In [7]:
BSV = BertSequenceVectorizer()
data['text_bert']=data['text'].progress_apply(lambda x: BSV.vectorize(x))
display(data[0:2])

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 14640/14640 [02:22<00:00, 102.85it/s]


,airline_sentiment,text,text_bert
0,1,@VirginAmerica What @dhepburn said.,"[-0.20783587, 0.53043973, -0.011864842, -0.062..."
1,2,@VirginAmerica plus you've added commercials t...,"[0.44031838, -0.06042881, -0.0036409795, -0.34..."


In [8]:
review=[]
for item in data['text_bert']:
    review+=[item]

dataX0=pd.DataFrame(review)
dataY=data['airline_sentiment']
print(dataX0.shape)
print(dataY.shape)

(14640, 768)
(14640,)


In [9]:
n=len(data)
N=list(range(n))
random.shuffle(N)

In [10]:
trainX0 = dataX0.iloc[N[0:(n//10)*3]].reset_index(drop=True)
trainY = dataY.iloc[N[0:(n//10)*3]].reset_index(drop=True)
testX0 = dataX0.iloc[N[(n//10)*3:(n//10)*4]].reset_index(drop=True)
testY = dataY.loc[N[(n//10)*3:(n//10)*4]].reset_index(drop=True)
print(trainX0.shape)
print(testX0.shape)

(4392, 768)
(1464, 768)


In [11]:
trainX = trainX0.iloc[:,0:200]
testX = testX0.iloc[:,0:200]
X = np.array(trainX)
y = np.array(trainY)

In [12]:
clf = lgb.LGBMRegressor(max_depth=3,n_estimators=1000,learning_rate=0.01)
ss = ShuffleSplit(n_splits=5,train_size=0.8,test_size=0.2,random_state=0) 
display(ss.split(X))

<generator object BaseShuffleSplit.split at 0x7fa395211250>

In [13]:
for train_index, test_index in ss.split(X): 
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    clf.fit(X_train, Y_train) 
    print(clf.score(X_test, Y_test))

0.515293716906648
0.45742912636954225
0.49972409978370746
0.49105171703716666
0.464722592293067


In [14]:
y_pred = clf.predict(testX)

In [15]:
PRED=[]
for pred in y_pred:
    PRED+=[int(round(pred,0))]   
ANS=list(testY)
print(PRED[0:5])
print(ANS[0:5])

[0, 0, 0, 0, 1]
[0, 2, 0, 0, 0]


In [16]:
from sklearn.metrics import classification_report
print(classification_report(ANS,PRED,target_names=Name,digits=4))

              precision    recall  f1-score   support

    negative     0.8577    0.7532    0.8021       928
     neutral     0.3659    0.6478    0.4677       318
    positive     0.8372    0.3303    0.4737       218

    accuracy                         0.6673      1464
   macro avg     0.6869    0.5771    0.5811      1464
weighted avg     0.7478    0.6673    0.6805      1464

